# NewsAPI.org-4 Data Collection
- Goal: Find top news by day
- Incrementally-updated political news will be compared against twitter and tiktok activity to asses responses to political events in the United States.

## Version 4: Counts words in title, not in body text of article + changed db connection info

In [1654]:
# Import necessary modules
import pandas as pd
from pandas.io import sql
import json
import requests
from datetime import date, timedelta
# from bs4 import BeautifulSoup
import numpy as np
from sqlalchemy import create_engine
import logging
import psycopg2
from psycopg2 import Error
import configparser


In [1655]:
c = configparser.ConfigParser()
c.read('sm_config.ini')

# config credentials
host = c['PostgreSQLdb']['host']
username = c['PostgreSQLdb']['user']
password = c['PostgreSQLdb']['password']
db = c['PostgreSQLdb']['database']


['sm_config.ini']

In [1639]:
# configure logger
logging.basicConfig(filename='news.log', filemode='w',
                    format=f'%(asctime)s - %(levelname)s - %(message)s')


In [1640]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = psycopg2.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        logging.info("PostgreSQL Database connection successful")
    except Error as err:
        logging.error(f"Error: '{err}'")

    return connection


In [1641]:
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        logging.info("Database created successfully")
    except Error as err:
        logging.error(f"Error: '{err}'")


In [1642]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = psycopg2.connect(
            host=host_name,
            user=user_name,
            password=user_password,
            database=db_name
        )
        # cursor = connection.cursor()
        logging.info("PostgreSQL Database connection successful")
    except Error as err:
        logging.error(f"Error: '{err}'")

    return connection


In [1643]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        logging.info("Query successful")
    except Error as err:
        print(f"Error: '{err}'")


In [1644]:
# connect to server
server = create_server_connection(host, username, password)

# create sm_news database
create_database_query = """
    CREATE DATABASE IF NOT EXISTS sm_news; 
"""
# create_database(server, create_database_query)


In [1645]:

# create necessary tables
create_article_table = """
CREATE TABLE IF NOT EXISTS articles (
    publishedAt DATE,
    title VARCHAR PRIMARY KEY,
    author VARCHAR,
    url TEXT, 
    keyWords VARCHAR
    );
"""
# CREATE INDEX index ON articles(publishedAt);
create_political_event_table = """
CREATE TABLE IF NOT EXISTS event (
    eventID VARCHAR PRIMARY KEY,
    startDate DATE,
    name VARCHAR NOT NULL,
    description VARCHAR NOT NULL,
    keyWords VARCHAR
    );
"""
create_tweets_table = """
CREATE TABLE IF NOT EXISTS tweets (
    tweet_id INT PRIMARY KEY,
    publishedAt DATE NOT NULL,
    userID INT NOT NULL,
    tweet VARCHAR NOT NULL,
    location VARCHAR NOT NULL, 
    tags VARCHAR NOT NULL
    );
"""
create_tiktoks_table = """
CREATE TABLE IF NOT EXISTS tiktoks (
    postID INT PRIMARY KEY,
    createTime DATE NOT NULL,
    description VARCHAR NOT NULL,
    musicID VARCHAR NOT NULL,
    tags VARCHAR NOT NULL,
    FOREIGN KEY(songID) REFERENCES tiktok_music(songID),
    FOREIGN KEY(soundID) REFERENCES tiktok_sounds(soundID),
    FOREIGN KEY(userID) REFERENCES users(userID)
    );
"""
create_tiktok_sounds_table = """
CREATE TABLE IF NOT EXISTS tiktok_sounds (
    soundID INT PRIMARY KEY,
    soundTitle VARCHAR,
    isOriginal BOOLEAN
    );
"""
create_tiktok_music_table = """
CREATE TABLE IF NOT EXISTS tiktok_music (
    songID INT PRIMARY KEY,
    songTitle VARCHAR NOT NULL
    );
"""
create_tiktok_stats_table = """
CREATE TABLE IF NOT EXISTS tiktok_stats (
    FOREIGN KEY(postID) REFERENCES tiktoks(postID),
    shareCount INT,
    commentCount INT,
    playCount INT,
    diggCount INT
    );
"""

create_tiktok_tags_table = """
CREATE TABLE IF NOT EXISTS tiktok_tags (
    tagID INT PRIMARY KEY,
    tag_name VARCHAR NOT NULL 
    );
"""
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
    userID INT PRIMARY KEY,
    username VARCHAR NOT NULL,
    user_bio VARCHAR NOT NULL
    );
"""
delete_bad_data = """
DELETE FROM articles
    WHERE publishedAt IS NULL;
"""


In [1646]:

# connect to db
connection = create_db_connection(host, username, password, db)

# execute defined queries to create db tables
execute_query(connection, create_article_table)
execute_query(connection, create_tweets_table)
execute_query(connection, create_political_event_table)
execute_query(connection, create_tiktok_sounds_table)
execute_query(connection, create_tiktok_music_table)

execute_query(connection, create_tiktok_stats_table)  # not running?
execute_query(connection, create_tiktok_tags_table)
execute_query(connection, create_tiktoks_table)


Error: 'relation "tiktoks" does not exist
'
Error: 'current transaction is aborted, commands ignored until end of transaction block
'
Error: 'current transaction is aborted, commands ignored until end of transaction block
'


In [1647]:
# read query
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")


In [1648]:
pop_news = []


def request_pop_news():

    params = {
        'q': ['politics' or 'political' or 'law' or 'legal' or 'policy'],
        'from': {date.today() - timedelta(days=3)},
        'to': {date.today},
        'language': 'en',
        'sort_by': 'popularity'
    }

    headers = {
        'X-Api-Key': c['newsAuth']['api_key']
    }

    url = 'https://newsapi.org/v2/everything'

    # response as JSON dict
    response = requests.get(url, params=params, headers=headers).json()

    with open('pop_news.json', 'w') as f:
        # write results to JSON file
        json.dump(response, f)

    with open('pop_news.json', 'r') as file:
        # create Python list object from JSON
        pop_news_json = file.read().split("\n")

        for story in pop_news_json:
            pop_obj = json.loads(story)

            if 'title' in pop_obj:
                pop_obj['title'] = pop_obj['articles']['title']
            if 'author' in pop_obj:
                pop_obj['author'] = pop_obj['articles']['author']
            if 'url' in pop_obj:
                pop_obj['url'] = pop_obj['articles']['url']
            if 'publishedAt' in pop_obj:
                pop_obj['publishedAt'] = pop_obj['articles']['publishedAt']

            # add info to pop_news dict
            pop_news.append(pop_obj)

        return pop_news


# call function
request_pop_news()

# load function call results into Pandas dataframe

# flatten data to dataframe
pop_news = pd.json_normalize(pop_news, record_path=['articles'])
pop_news_df = pd.DataFrame(
    pop_news, columns=['title', 'author', 'url', 'publishedAt'])
pop_news_df = pop_news_df.dropna(axis=0, how='any')
pop_news_df.head()


,title,author,url,publishedAt
0,Diving-Mexico's Marrufo jumps from the springb...,"Richa Naidu,Yuki Nitta",https://www.reuters.com/lifestyle/sports/divin...,2021-08-03T05:41:00Z
1,Politics this week,The Economist,https://www.economist.com/the-world-this-week/...,2021-08-05T15:25:29Z
2,Who is the real Sajid Javid? Politics Weekly p...,Presented by Jessica Elgot. With Robert Halfon...,https://www.theguardian.com/politics/audio/202...,2021-08-04T04:00:15Z
3,10 Things in Politics: Buttigieg mafia becomin...,bgriffiths@insider.com (Brent D. Griffiths),https://www.businessinsider.com/10-things-in-p...,2021-08-05T09:24:23Z
4,"How Trump reeled in over $100 million, televan...",Opinion by Michael D'Antonio,https://www.cnn.com/2021/08/03/opinions/trump-...,2021-08-03T19:16:26Z


In [1649]:
# get top headlines
top_headlines = []


def get_top_headlines():
    params = {
        "language": "en",
        "country": "us"
    }
    headers = {
        "X-Api-Key": c['newsAuth']['api_key']
    }
    url = "https://newsapi.org/v2/top-headlines"

    response = requests.get(
        url, params=params, headers=headers).json()  # response JSON dict

    with open("top_headlines.json", "w") as f:
        # write results to JSON file
        json.dump(response, f)

    with open("top_headlines.json", "r") as file:
        # create Python object from JSON
        top_headlines_json = file.read().split("\n")

        for story in top_headlines_json:
            story_obj = json.loads(story)

            if 'title' in story_obj:
                story_obj["title"] = story_obj["articles"]["title"]
            if 'author' in story_obj:
                story_obj["author"] = story_obj["articles"]["author"]
            if 'url' in story_obj:
                story_obj["url"] = story_obj["articles"]["url"]
            if 'publishedAt' in story_obj:
                story_obj["publishedAt"] = story_obj["articles"]["publishedAt"]

            # add info to top_headlines list/dict
            top_headlines.append(story_obj)

        return top_headlines


# flatten data to dataframe
get_top_headlines()
top_headlines = pd.json_normalize(top_headlines, record_path=['articles'])
top_headlines_df = pd.DataFrame(
    top_headlines, columns=["title", "author", "url", "publishedAt"])
top_headlines_df = top_headlines_df.dropna(axis=0, how='any')

top_headlines_df.head(20)



,title,author,url,publishedAt
0,DOJ opens investigation into how Phoenix Polic...,"Christina Carrega and Devan Cole, CNN",https://www.cnn.com/2021/08/05/politics/phoeni...,2021-08-05T20:14:00Z
1,"Richard Trumka, President Of The AFL-CIO, Dies...",Benjamin Swasey,https://www.npr.org/2021/08/05/1025136446/powe...,2021-08-05T20:08:00Z
2,Jan. 6 select panel takes over House probe of ...,"Betsy Woodruff Swan, Nicholas Wu",https://www.politico.com/news/2021/08/05/jan-6...,2021-08-05T19:54:37Z
3,Soccer superstar Lionel Messi leaving FC Barce...,David K. Li,https://www.nbcnews.com/news/sports/soccer-sup...,2021-08-05T19:25:00Z
4,Only one USWNT player didn't kneel ahead of Ol...,Gabrielle Fonrouge,https://nypost.com/2021/08/05/only-one-uswnt-p...,2021-08-05T19:14:00Z
5,New Google Nest Cams can record video without ...,Ron Amadeo,https://arstechnica.com/gadgets/2021/08/new-go...,2021-08-05T19:12:23Z
6,Apple confirms it will begin scanning iCloud P...,Zack Whittaker,http://techcrunch.com/2021/08/05/apple-icloud-...,2021-08-05T19:00:28Z
7,Unvaccinated Minnesota Vikings QB Kirk Cousins...,Courtney Cronin,https://www.espn.com/nfl/story/_/id/31966319/u...,2021-08-05T18:56:34Z
8,Tesla just got snubbed by Biden's electric veh...,"Chris Isidore, CNN Business",https://www.cnn.com/2021/08/05/business/tesla-...,2021-08-05T18:50:00Z
9,A $5800 whisky bottle Japan gave to Mike Pompe...,"Claire Parker, John Hudson",https://www.washingtonpost.com/world/2021/08/0...,2021-08-05T18:45:00Z


In [1650]:
all_news = pd.concat([top_headlines_df, pop_news_df])
all_news['publishedAt'] = pd.to_datetime(all_news['publishedAt'])

all_news.info()
all_news.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 37 entries, 0 to 19
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   title        37 non-null     object             
 1   author       37 non-null     object             
 2   url          37 non-null     object             
 3   publishedAt  37 non-null     datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(3)
memory usage: 1.4+ KB


,title,author,url,publishedAt
0,DOJ opens investigation into how Phoenix Polic...,"Christina Carrega and Devan Cole, CNN",https://www.cnn.com/2021/08/05/politics/phoeni...,2021-08-05 20:14:00+00:00
1,"Richard Trumka, President Of The AFL-CIO, Dies...",Benjamin Swasey,https://www.npr.org/2021/08/05/1025136446/powe...,2021-08-05 20:08:00+00:00
2,Jan. 6 select panel takes over House probe of ...,"Betsy Woodruff Swan, Nicholas Wu",https://www.politico.com/news/2021/08/05/jan-6...,2021-08-05 19:54:37+00:00
3,Soccer superstar Lionel Messi leaving FC Barce...,David K. Li,https://www.nbcnews.com/news/sports/soccer-sup...,2021-08-05 19:25:00+00:00
4,Only one USWNT player didn't kneel ahead of Ol...,Gabrielle Fonrouge,https://nypost.com/2021/08/05/only-one-uswnt-p...,2021-08-05 19:14:00+00:00


In [1651]:
def execute_mogrify(conn, df, table):
    """
    Using cursor.mogrify() to build the bulk insert query
    then cursor.execute() to execute the query
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify("(%s,%s,%s,%s)", tup).decode('utf8')
              for tup in tuples]
    # if not publishedAt, delete record
    query = "INSERT INTO %s(%s) VALUES" % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        logging.error("Error: %s" % error)
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        conn.close()
        return 1
    logging.info("execute_mogrify() done")
    cursor.close()
    conn.close()


In [1652]:
connection = create_db_connection("localhost", "postgres", "521368", "sm_news")
execute_mogrify(connection, all_news, 'articles')



Error: duplicate key value violates unique constraint "articles_pkey"
DETAIL:  Key (title)=(DOJ opens investigation into how Phoenix Police Department treats city's homeless residents - CNN) already exists.



1